## <font color="blue"> Data</font>

In [11]:
using DataFrames
using JWAS.Datasets
using JWAS.MT

In [12]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### Genetic covariance matrix and residual covariance matrix

In [13]:
R      = [10.0 0.0
           0.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [14]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


## <font color="blue">3. mulitiple traits with marker effects</font> 

#### set up model equations

In [15]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [16]:
model3    = MT.build_model(model_equations,R);

In [17]:
MT.set_covariate(model3,"age")

In [18]:
MT.add_markers(model3,genofile,G,separator=',',header=true);

The delimiters in file /Users/haocheng/.julia/v0.4/JWAS/src/5.Datasets/src/../data/testMT/genotype.txt is ,  .


In [19]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out2 = MT.runMCMC(model3,data,Pi=Pi,chain_length=50,constraint=true,
                  printout_frequency=10,methods="BayesC",output_marker_effects_frequency=1);

posterior means at sample: 10
Residual covariance matrix: 
[66.45262094074693 0.0
 0.0 0.4551138307758551]
Marker effects covariance matrix: 
[50.80121935995164 19.01520010653735
 19.01520010653735 9.031630183677091]

π: 
Dict([0.0,1.0]=>0.2255089183873541,[1.0,0.0]=>0.4519159523814974,[1.0,1.0]=>0.19783548071334559,[0.0,0.0]=>0.12473964851780295)
posterior means at sample: 20
Residual covariance matrix: 
[35.59024236830428 0.0
 0.0 0.4708892625754071]
Marker effects covariance matrix: 
[69.17839269755845 30.370361152377356
 30.370361152377356 15.747950097868621]

π: 
Dict([0.0,1.0]=>0.2664307400479557,[1.0,0.0]=>0.45091188364115387,[1.0,1.0]=>0.18963399596865774,[0.0,0.0]=>0.09302338034223266)
posterior means at sample: 30
Residual covariance matrix: 
[25.559020080216435 0.0
 0.0 0.5828569570900948]
Marker effects covariance matrix: 
[73.3728942087813 27.446134955367903
 27.446134955367903 12.701487549218353]

π: 
Dict([0.0,1.0]=>0.28357796698642035,[1.0,0.0]=>0.45273918945215125,[1.0

In [10]:
using JWAS.QTL

In [21]:
file1="marker_effects_BW_2016-07-26T14:39:03.txt"
file2="marker_effects_CW_2016-07-26T14:39:03.txt";

In [22]:
a,b,c=get_additive_genetic_variances(model3,file1,file2);

In [23]:
?get_additive_genetic_variances

search: get_additive_genetic_variances



```
get_additive_genetic_variances(model::MME,files...)
```

  * calculate distributions of additive genetic variances with output_marker_effects **files**
  * return samples, mean, variance for additive genetic variances


In [26]:
get_breeding_values(model3,file1,file2)

2-element Array{Any,1}:
 4×3 DataFrames.DataFrame
│ Row │ ID   │ EBV      │ PEV     │
├─────┼──────┼──────────┼─────────┤
│ 1   │ "S1" │ -11.0744 │ 11.0705 │
│ 2   │ "D1" │ 1.57389  │ 11.9758 │
│ 3   │ "O1" │ 23.223   │ 25.7696 │
│ 4   │ "O3" │ -13.7226 │ 8.3652  │            
 4×3 DataFrames.DataFrame
│ Row │ ID   │ EBV       │ PEV      │
├─────┼──────┼───────────┼──────────┤
│ 1   │ "S1" │ -0.183236 │ 1.20921  │
│ 2   │ "D1" │ 1.2146    │ 1.26763  │
│ 3   │ "O1" │ 4.149     │ 1.55523  │
│ 4   │ "O3" │ -5.18036  │ 0.933692 │